# Libraries

In [6]:
import os
import cv2
from mtcnn import MTCNN
import tensorflow
from mtcnn import MTCNN
import cv2
import os
import numpy as np
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

#Augmentation
import imgaug.augmenters as iaa

# Alignment Class [MTCNN backed]

The code below will input the image. Then it detects the face or faces in the image. It then selects the largest face (most pixels in it) and detects the centers of the left and right eye. It then determines the angle of the line joining the eye centers. This angle is then feed into a function that rotates the image so the line is paralllel to the x axis. Then the rotatedimage is cropped to return the largest face and the image is stored in the working/dir/car. MTCNN is the most acurate face cropper I have found but unfortunately it is very slow. 

First allignment of the dataset is done using MTCNN model, alignment basically means to detect a face and then using trignometric functions and landmarks of the face, the face is rotated in a way that eyes are aligned horizaontally, all of this done to make all the images have same orientation and alignment helps achieve better accuracy also.

The width and height are optional arguments that can be used to resize all the processed image into a fixed size

In [7]:
class ImageAligner:
    def __init__(self, detector):
        self.detector = detector
    
    def align(self, img):
        data = self.detector.detect_faces(img)  # Use the detector to detect the faces in the image
        biggest = 0
        if data != []:  # If at least one face is detected
            for faces in data:
                box = faces['box']  # Get the bounding box coordinates of the face
                area = box[3] * box[2]  # Calculate the area of the bounding box
                if area > biggest:  # If this is the largest face so far, update the biggest area
                    biggest = area
                    bbox = box
                    keypoints = faces['keypoints']  # Get the keypoints of the face
                    left_eye = keypoints['left_eye']  # Get the coordinates of the left eye
                    right_eye = keypoints['right_eye']  # Get the coordinates of the right eye
            lx, ly = left_eye  # Unpack the coordinates of the left eye
            rx, ry = right_eye  # Unpack the coordinates of the right eye
            dx = rx - lx  # Calculate the difference in x-coordinates between the eyes
            dy = ry - ly  # Calculate the difference in y-coordinates between the eyes
            tan = dy / dx  # Calculate the tangent of the angle between the eyes
            theta = np.arctan(tan)  # Calculate the angle in radians
            theta = np.degrees(theta)  # Convert the angle to degrees
            img = self.rotate_bound(img, theta)  # Rotate the image by the calculated angle
            return (True, img)  # Return a tuple indicating success and the rotated image
        else:  # If no face is detected
            return (False, img)  # Return a tuple indicating failure and no image


    def crop_image(self, img):
        data = self.detector.detect_faces(img)
        biggest = 0
        if data != []:
            for faces in data:
                box = faces['box']
                area = box[3] * box[2]
                if area > biggest:
                    biggest = area
                    bbox = box
            bbox[0] = 0 if bbox[0] < 0 else bbox[0]
            bbox[1] = 0 if bbox[1] < 0 else bbox[1]
            img = img[bbox[1]: bbox[1] + bbox[3], bbox[0]: bbox[0] + bbox[2]]
            return (True, img)
        else:
            return (False, img)

    def rotate_bound(self, image, angle):
        (h, w) = image.shape[:2]
        (cX, cY) = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
        cos = np.abs(M[0, 0])
        sin = np.abs(M[0, 1])
        nW = int((h * sin) + (w * cos))
        nH = int((h * cos) + (w * sin))
        M[0, 2] += (nW / 2) - cX
        M[1, 2] += (nH / 2) - cY
        return cv2.warpAffine(image, M, (nW, nH))
    
    def align_db(self, main_folder, aligned_folder ,height=None, width=None):
        
        for subfolder in os.listdir(main_folder):
            subfolder_path = os.path.join(main_folder, subfolder)
            if os.path.isdir(subfolder_path):
                
                aligned_subfolder_path = os.path.join(aligned_folder, subfolder)
                if not os.path.exists(aligned_subfolder_path):
                    os.mkdir(aligned_subfolder_path)
                
                for image_name in os.listdir(subfolder_path):
                    image_path = os.path.join(subfolder_path, image_name)

                    aligned_image_path = os.path.join(aligned_subfolder_path, image_name)

                    img = cv2.imread(image_path)
                    success, img = self.align(img)
                    if success:
                        successT, img = self.crop_image(img)
                        if successT:
                            if height is not None and width is not None:
                                img = cv2.resize(img, (width, height))
                    cv2.imwrite(aligned_image_path, img)

# Inputs

In [9]:
# Set the path to database folder containing the images
db_folder = '../Databases/Original/ibad'

aligned_folder = '../Databases/Alligned/ibad'

if not os.path.exists(aligned_folder):
    os.makedirs(aligned_folder)

# Load the MTCNN model
detector = MTCNN()


image_aligner = ImageAligner(detector)
image_aligner.align_db(db_folder, aligned_folder, height=160, width=160)
# image_aligner.align_db(db_folder)

1/1 [==============================] - 0s 16ms/step


# Augmentation [imgaug backed]

The code defines a set of image augmentations that can be applied to images as a form of data augmentation. The augmentations are defined using the imgaug library, which is a library for image augmentation in Python.

The specific augmentations defined in the code are:

Affine rotation: rotates the image by a random angle between -20 and 20 degrees.
Brightness adjustment: multiplies the brightness of the image by a random factor between 0.5 and 1.5 and adds a random value between -30 and 30 to the brightness.
Gaussian blur: adds a Gaussian blur with a random sigma value between 0 and 1.5.
Additive Gaussian noise: adds Gaussian noise with a standard deviation up to 5% of the image intensity range.
Affine scaling: scales the image along the x and y axes by a random factor between 0.8 and 1.2.

These augmentations can be applied to input images to create additional training examples for machine learning models, which can improve the performance and robustness of the models. By applying these augmentations, the model can learn to recognize faces that have been rotated, scaled, or have different levels of brightness or noise.

In [12]:
# Define the augmentations
seq = iaa.Sequential([
    iaa.Affine(rotate=(-20, 20)), # rotate between -20 and 20 degrees
    iaa.MultiplyAndAddToBrightness(mul=(0.5, 1.5), add=(-30, 30)), # adjust brightness by up to +/-30
    iaa.GaussianBlur(sigma=(0, 1.5)), # add Gaussian blur with sigma between 0 and 1.5
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)), # add Gaussian noise with standard deviation up to 5% of image intensity range
    iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}), # scale image by up to 20%
])

# Inputs

In [ ]:
# Set the path to the main folder
main_folder = "../Misc/DroneFace/Height_Distance/database_n"

# Loop over every subfolder in the main folder
for folder in os.listdir(main_folder):
    folder_path = os.path.join(main_folder, folder)
    
    # Only process the subfolders that contain images
    if os.path.isdir(folder_path) and any(filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")) for filename in os.listdir(folder_path)):

        # Loop over every image in the subfolder
        for filename in os.listdir(folder_path):
            if filename.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
                
                # read image from file
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path)

                # Convert the image to a numpy array
                img_arr = np.array(image)

                # Augment the image 20 times
                images_aug = seq(images=np.tile(img_arr, (20, 1, 1, 1)))

                # Save the augmented images using OpenCV
                for i in range(20):
                    # Save the image
                    cv2.imwrite(image_path + f'augmented_{i}.jpg', images_aug[i])
